## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [39]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['/home/nirojasvar/random_sast/ExperimentationRSAST',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python310.zip',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/lib-dynload',
 '',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nirojasvar/random_sast/cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/cd_diagram',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nir

In [40]:
from sast import *
import pandas as pd
import researchpy
import math
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

### Reading Datasets

In [41]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [42]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'SAST', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [43]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

directories=[]
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST')
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_All')
#directories.append(os.getcwd()+'/ResultsRsast/Server16_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/results_accuracy_per_ds')
#directories.append(os.getcwd()+'/results_accuracy_per_ds_10000')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


/tmp/ipykernel_55898/2832072796.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_55898/2832072796.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_55898/2832072796.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_55898/2832072796.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_55898/2832072796.py:29: FutureWarning: The default value of

In [44]:
df_used_sast=pd.read_excel("DataSetsUCLASummary.xlsx", dtype=str)
df_used_sast.columns

Index(['USED SAST', 'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'ID', 'Name', 'Type', 'Train ', 'Test ', 'Class', 'Length',
       'RSAST (10,10) WORST SAST?', 'O(|c|nm²)', 'O(|c|nm³)',
       'Approx Time (hours)', 'L25', 'L50', 'L75', 'ED (w=0)',
       'DTW (learned_w) ', 'DTW (w=100)', 'Default rate', 'Data donor/editor',
       '50/L', 'Unnamed: 24'],
      dtype='object')

In [45]:
df_used_sast=df_used_sast[['Name', 'Type', 'Train ', 'Test ', 'Class', 'Length','USED SAST']]
df_used_sast=df_used_sast[df_used_sast['USED SAST']=='Y']
df_used_sast

,Name,Type,Train,Test,Class,Length,USED SAST
0,SmoothSubspace,Simulated,150,150,3,15,Y
1,Chinatown,Traffic,20,343,2,24,Y
2,ItalyPowerDemand,Sensor,67,1029,2,24,Y
3,MelbournePedestrian,Traffic,1194,2439,10,24,Y
4,Crop,Image,7200,16800,24,46,Y
...,...,...,...,...,...,...,...
77,BeetleFly,Image,20,20,2,512,Y
78,BirdChicken,Image,20,20,2,512,Y
79,Herring,Image,64,64,2,512,Y
80,Earthquakes,Sensor,322,139,2,512,Y


In [46]:
pv_all=pd.pivot_table(df_result,index='dataset_name',values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
ds_complete
df_result=df_result[df_result.dataset_name.isin(ds_complete)]

In [47]:
#get ds tested overall
print("Total DS:" + str(len(df_result['dataset_name'].unique())))

Total DS:30


In [48]:
#get ds tested overall
df_result['dataset_name'].unique()

array(['MiddlePhalanxOutlineAgeGroup', 'DistalPhalanxOutlineAgeGroup',
       'ECG200', 'ItalyPowerDemand', 'TwoLeadECG',
       'ProximalPhalanxOutlineCorrect', 'WordSynonyms',
       'SonyAIBORobotSurface2', 'ProximalPhalanxOutlineAgeGroup',
       'CricketY', 'FiftyWords', 'MiddlePhalanxOutlineCorrect',
       'ProximalPhalanxTW', 'SonyAIBORobotSurface1',
       'PhalangesOutlinesCorrect', 'SmoothSubspace', 'CricketZ',
       'CricketX', 'ElectricDevices', 'ECGFiveDays', 'Chinatown', 'Adiac',
       'DistalPhalanxOutlineCorrect', 'Coffee', 'MiddlePhalanxTW',
       'MoteStrain', 'Crop', 'SyntheticControl', 'DistalPhalanxTW',
       'MedicalImages'], dtype=object)

In [49]:
#get ds tested overall
print("Total Param Combination:" + str(len(df_result['classifier_name'].unique())))

Total Param Combination:16


In [50]:
#get len methods of generated datasets
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [51]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

In [52]:
pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index()

classifier_name,dataset_name,ACF&PACF: n_random_points=10 nb_inst_per_class=1,ACF&PACF: n_random_points=10 nb_inst_per_class=10,ACF&PACF: n_random_points=30 nb_inst_per_class=1,ACF&PACF: n_random_points=30 nb_inst_per_class=10,Max PACF: n_random_points=10 nb_inst_per_class=1,Max PACF: n_random_points=10 nb_inst_per_class=10,Max PACF: n_random_points=30 nb_inst_per_class=1,Max PACF: n_random_points=30 nb_inst_per_class=10,None: n_random_points=10 nb_inst_per_class=1,None: n_random_points=10 nb_inst_per_class=10,None: n_random_points=30 nb_inst_per_class=1,None: n_random_points=30 nb_inst_per_class=10,all: n_random_points=10 nb_inst_per_class=1,all: n_random_points=10 nb_inst_per_class=10,all: n_random_points=30 nb_inst_per_class=1,all: n_random_points=30 nb_inst_per_class=10
0,Adiac,0.631714,0.576982,0.606138,0.569309,0.701790,0.497698,0.448082,0.502302,0.705882,0.623529,0.588235,0.639898,0.595396,0.549361,0.559079,0.547315
1,Chinatown,0.956851,0.976093,0.961516,0.982507,0.941691,0.982507,0.945190,0.985423,0.917201,0.963848,0.754519,0.966181,0.971429,0.985423,0.977259,0.985423
2,Coffee,0.978571,1.000000,1.000000,1.000000,0.928571,0.935714,0.928571,0.935714,0.914286,0.992857,0.935714,1.000000,1.000000,1.000000,1.000000,1.000000
3,CricketX,0.729744,0.796410,0.745641,0.798462,0.676923,0.732821,0.703077,0.783077,0.624103,0.711795,0.668205,0.745641,0.734359,0.770256,0.729231,0.762564
4,CricketY,0.688205,0.764103,0.711795,0.771282,0.641538,0.698974,0.666154,0.732308,0.590769,0.692308,0.647179,0.723077,0.722051,0.744615,0.720513,0.755897
5,CricketZ,0.702051,0.783077,0.739487,0.785641,0.665128,0.734359,0.677436,0.747692,0.645128,0.704615,0.644615,0.744103,0.720000,0.756923,0.721026,0.764103
6,Crop,0.699976,0.630286,0.712976,0.707560,0.668595,0.723500,0.692310,0.728298,0.680393,0.736821,0.695214,0.740976,0.633786,0.744417,0.702488,0.750619
7,DistalPhalanxOutlineAgeGroup,0.751079,0.723741,0.751079,0.720863,0.741007,0.748201,0.748201,0.736691,0.717986,0.741007,0.723741,0.758273,0.735252,0.722302,0.723741,0.715108
8,DistalPhalanxOutlineCorrect,0.736232,0.726087,0.723913,0.737681,0.725362,0.723188,0.729710,0.711594,0.725362,0.765942,0.712319,0.759420,0.727536,0.730435,0.701449,0.724638
9,DistalPhalanxTW,0.651799,0.618705,0.640288,0.614388,0.673381,0.640288,0.674820,0.605755,0.669065,0.647482,0.666187,0.643165,0.621583,0.627338,0.617266,0.620144


In [53]:
"""
df_tunned_rsast=df_used_sast.merge(pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index(),how='right',right_on="dataset_name",left_on="Name").round(2)[['Name', 'ACF&PACF: n_random_points=30 nb_inst_per_class=10','ACF&PACF: n_random_points=30 nb_inst_per_class=10','ACF&PACF: n_random_points=30 nb_inst_per_class=1','Type', 'Train ', 'Test ', 'Class', 'Length']]
df_tunned_rsast.sort_values("Name",inplace=True)
df_tunned_rsast.to_excel('ds_runned_rsast.xlsx')
"""

'\ndf_tunned_rsast=df_used_sast.merge(pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index(),how=\'right\',right_on="dataset_name",left_on="Name").round(2)[[\'Name\', \'ACF&PACF: n_random_points=30 nb_inst_per_class=10\',\'ACF&PACF: n_random_points=30 nb_inst_per_class=10\',\'ACF&PACF: n_random_points=30 nb_inst_per_class=1\',\'Type\', \'Train \', \'Test \', \'Class\', \'Length\']]\ndf_tunned_rsast.sort_values("Name",inplace=True)\ndf_tunned_rsast.to_excel(\'ds_runned_rsast.xlsx\')\n'

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [54]:
filter_param=df_result

#filter_param=filter_param[filter_param.len_method=='ACF&PACF']
filter_param.dataset_name.unique()

array(['MiddlePhalanxOutlineAgeGroup', 'DistalPhalanxOutlineAgeGroup',
       'ECG200', 'ItalyPowerDemand', 'TwoLeadECG',
       'ProximalPhalanxOutlineCorrect', 'WordSynonyms',
       'SonyAIBORobotSurface2', 'ProximalPhalanxOutlineAgeGroup',
       'CricketY', 'FiftyWords', 'MiddlePhalanxOutlineCorrect',
       'ProximalPhalanxTW', 'SonyAIBORobotSurface1',
       'PhalangesOutlinesCorrect', 'SmoothSubspace', 'CricketZ',
       'CricketX', 'ElectricDevices', 'ECGFiveDays', 'Chinatown', 'Adiac',
       'DistalPhalanxOutlineCorrect', 'Coffee', 'MiddlePhalanxTW',
       'MoteStrain', 'Crop', 'SyntheticControl', 'DistalPhalanxTW',
       'MedicalImages'], dtype=object)

In [55]:
filter_param.rpoint.unique()

array(['10', '30'], dtype=object)

In [56]:
#filter_param=filter_param[filter_param.rpoint=='half_len']

In [57]:
filter_param.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [58]:
#filter_param=filter_param[filter_param.nb_per_class=='30']

In [59]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="SAST"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df


,dataset_name,ACF&PACF,Max PACF,None,all,score,method
0,Adiac,0.596036,0.537468,0.639386,0.562788,0.68,SAST
1,Chinatown,0.969242,0.963703,0.900437,0.979883,0.96,SAST
2,Coffee,0.994643,0.932143,0.960714,1.000000,1.00,SAST
3,CricketX,0.767564,0.723974,0.687436,0.749103,0.77,SAST
4,CricketY,0.733846,0.684744,0.663333,0.735769,0.74,SAST
5,CricketZ,0.752564,0.706154,0.684615,0.740513,0.77,SAST
6,Crop,0.687699,0.703176,0.713351,0.707827,0.73,SAST
7,DistalPhalanxOutlineAgeGroup,0.736691,0.743525,0.735252,0.724101,0.76,SAST
8,DistalPhalanxOutlineCorrect,0.730978,0.722464,0.740761,0.721014,0.74,SAST
9,DistalPhalanxTW,0.631295,0.648561,0.656475,0.621583,0.67,SAST


In [60]:
merged_df.describe()

,ACF&PACF,Max PACF,None,all,score
count,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.773125,0.749329,0.754427,0.776815,0.796667
std,0.127521,0.127835,0.121294,0.149705,0.125020
min,0.530844,0.537468,0.537662,0.441830,0.530000
25%,0.698519,0.657607,0.658189,0.711124,0.732500
50%,0.780905,0.725819,0.738006,0.764387,0.775000
75%,0.860347,0.837744,0.819381,0.872312,0.865000
max,0.994643,0.963703,0.968641,1.000000,1.000000


In [61]:

# Summary statistics for a Series (single variable)

summ_ds=filter_param
summ_ds=researchpy.summary_cont(summ_ds.groupby(['dataset_name'])['accuracy'], conf = 0.95)
summ_ds

,N,Mean,SD,SE,95% Conf.,Interval
dataset_name,,,,,,
Adiac,80,0.5839,0.0683,0.0076,0.5687,0.5991
Chinatown,80,0.9533,0.0754,0.0084,0.9365,0.9701
Coffee,80,0.9719,0.0443,0.0049,0.9620,0.9817
CricketX,80,0.7320,0.0492,0.0055,0.7211,0.7430
CricketY,80,0.7044,0.0513,0.0057,0.6930,0.7158
CricketZ,80,0.7210,0.0471,0.0053,0.7105,0.7314
Crop,80,0.7030,0.0354,0.0040,0.6951,0.7109
DistalPhalanxOutlineAgeGroup,80,0.7349,0.0213,0.0024,0.7302,0.7396
DistalPhalanxOutlineCorrect,80,0.7288,0.0235,0.0026,0.7236,0.7340


In [62]:
stats = filter_param.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std   ci95_hi   ci95_lo
len_method                                               
ACF&PACF    0.773125    600  0.134881  0.783918  0.762332
Max PACF    0.749329    600  0.139197  0.760467  0.738191
None        0.754427    600  0.133229  0.765088  0.743767
all         0.776815    600  0.150903  0.788889  0.764740


#### Generate Boxplot Tunning Hyperparameter

In [63]:
df_result.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [64]:
df_result.len_method.unique()

array(['ACF&PACF', 'Max PACF', 'None', 'all'], dtype=object)

In [65]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_result.nb_per_class.unique()):

    for len_m in df_result.len_method.unique():
        
        
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.accuracy)
        min_bx=min(df_to_cd.accuracy)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_acc/boxplot_acc'+len_m+ints+'.png')




In [66]:
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [67]:
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
            print("ds:"+ds)
            print("ints:"+str(ints))
            print("len_m:"+len_m)

            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.accuracy)
            min_bx=min(df_to_cd.accuracy)
            
            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]    

            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            
            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_acc_per_ds/boxplot_acc'+len_m+ints+'_'+ds+'.png')

ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:ACF&PACF
ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:Max PACF
ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:None
ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:all
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:ACF&PACF
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:Max PACF
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:None
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:all
ds:DistalPhalanxOutlineAgeGroup
ints:1
len_m:ACF&PACF


/tmp/ipykernel_55898/3028681140.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


ds:DistalPhalanxOutlineAgeGroup
ints:1
len_m:Max PACF
ds:DistalPhalanxOutlineAgeGroup
ints:1
len_m:None
ds:DistalPhalanxOutlineAgeGroup
ints:1
len_m:all
ds:DistalPhalanxOutlineAgeGroup
ints:10
len_m:ACF&PACF
ds:DistalPhalanxOutlineAgeGroup
ints:10
len_m:Max PACF
ds:DistalPhalanxOutlineAgeGroup
ints:10
len_m:None
ds:DistalPhalanxOutlineAgeGroup
ints:10
len_m:all
ds:ECG200
ints:1
len_m:ACF&PACF
ds:ECG200
ints:1
len_m:Max PACF
ds:ECG200
ints:1
len_m:None
ds:ECG200
ints:1
len_m:all
ds:ECG200
ints:10
len_m:ACF&PACF
ds:ECG200
ints:10
len_m:Max PACF
ds:ECG200
ints:10
len_m:None
ds:ECG200
ints:10
len_m:all
ds:ItalyPowerDemand
ints:1
len_m:ACF&PACF
ds:ItalyPowerDemand
ints:1
len_m:Max PACF
ds:ItalyPowerDemand
ints:1
len_m:None
ds:ItalyPowerDemand
ints:1
len_m:all
ds:ItalyPowerDemand
ints:10
len_m:ACF&PACF
ds:ItalyPowerDemand
ints:10
len_m:Max PACF
ds:ItalyPowerDemand
ints:10
len_m:None
ds:ItalyPowerDemand
ints:10
len_m:all
ds:TwoLeadECG
ints:1
len_m:ACF&PACF
ds:TwoLeadECG
ints:1
len_m:Max PACF


/tmp/ipykernel_55898/3028681140.py:32: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(min_bx,max_bx)


ds:Coffee
ints:10
len_m:Max PACF
ds:Coffee
ints:10
len_m:None
ds:Coffee
ints:10
len_m:all


/tmp/ipykernel_55898/3028681140.py:32: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(min_bx,max_bx)


ds:MiddlePhalanxTW
ints:1
len_m:ACF&PACF
ds:MiddlePhalanxTW
ints:1
len_m:Max PACF
ds:MiddlePhalanxTW
ints:1
len_m:None
ds:MiddlePhalanxTW
ints:1
len_m:all
ds:MiddlePhalanxTW
ints:10
len_m:ACF&PACF
ds:MiddlePhalanxTW
ints:10
len_m:Max PACF
ds:MiddlePhalanxTW
ints:10
len_m:None
ds:MiddlePhalanxTW
ints:10
len_m:all
ds:MoteStrain
ints:1
len_m:ACF&PACF
ds:MoteStrain
ints:1
len_m:Max PACF
ds:MoteStrain
ints:1
len_m:None
ds:MoteStrain
ints:1
len_m:all
ds:MoteStrain
ints:10
len_m:ACF&PACF
ds:MoteStrain
ints:10
len_m:Max PACF
ds:MoteStrain
ints:10
len_m:None
ds:MoteStrain
ints:10
len_m:all
ds:Crop
ints:1
len_m:ACF&PACF
ds:Crop
ints:1
len_m:Max PACF
ds:Crop
ints:1
len_m:None
ds:Crop
ints:1
len_m:all
ds:Crop
ints:10
len_m:ACF&PACF
ds:Crop
ints:10
len_m:Max PACF
ds:Crop
ints:10
len_m:None
ds:Crop
ints:10
len_m:all
ds:SyntheticControl
ints:1
len_m:ACF&PACF
ds:SyntheticControl
ints:1
len_m:Max PACF
ds:SyntheticControl
ints:1
len_m:None
ds:SyntheticControl
ints:1
len_m:all
ds:SyntheticControl
ints:10

In [68]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_result.nb_per_class.unique()):
    for len_m in df_result.len_method.unique():
        
        
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
        df_to_cd["time"]=df_to_cd["time"]/60
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.time)
        min_bx=min(df_to_cd.time)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_time/boxplot_time'+len_m+ints+'.png')




/tmp/ipykernel_55898/4020300904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
/tmp/ipykernel_55898/4020300904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd["time"]/60
/tmp/ipykernel_55898/4020300904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [69]:
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
        


            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.time)
            min_bx=min(df_to_cd.time)

            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]


            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            

            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_time_per_ds/boxplot_time'+len_m+ints+'_'+ds+'.png')


#### Generate CD Diagram Tunning Hyperparameter

In [70]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_perf.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [71]:

df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')].classifier_name.unique()

array(['all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [72]:


from cd_function import *
#generate hyperparameter tuning cd diagrams

best_comb_by_method=[]
best=""
for len_m in df_perf.len_method.unique():
   df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')]
   draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison", fname='images_cd_diagram/cd-diagram_'+len_m+'.png')
   _, average_ranks, _ = wilcoxon_holm(df_perf=df_to_cd)
   min_rank= min(average_ranks)
   average_ranks=pd.DataFrame(average_ranks)
   best=average_ranks[average_ranks[0]==min_rank][0].index
   best_comb_by_method.append(best[0])



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families 

{0: 'ACF&PACF: n_random_points=10 nb_inst_per_class=1', 1: 'ACF&PACF: n_random_points=30 nb_inst_per_class=1', 2: 'ACF&PACF: n_random_points=10 nb_inst_per_class=10', 3: 'ACF&PACF: n_random_points=30 nb_inst_per_class=10'}
[0, 1, 2, 3]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

{0: 'Max PACF: n_random_points=10 nb_inst_per_class=1', 1: 'Max PACF: n_random_points=30 nb_inst_per_class=1', 2: 'Max PACF: n_random_points=10 nb_inst_per_class=10', 3: 'Max PACF: n_random_points=30 nb_inst_per_class=10'}


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
findfont: Gene

{0: 'None: n_random_points=10 nb_inst_per_class=1', 1: 'None: n_random_points=30 nb_inst_per_class=1', 2: 'None: n_random_points=10 nb_inst_per_class=10', 3: 'None: n_random_points=30 nb_inst_per_class=10'}
[0, 1]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

{0: 'all: n_random_points=10 nb_inst_per_class=1', 1: 'all: n_random_points=30 nb_inst_per_class=1', 2: 'all: n_random_points=30 nb_inst_per_class=10', 3: 'all: n_random_points=10 nb_inst_per_class=10'}
[0, 1, 2, 3]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

In [73]:
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10',
 'all: n_random_points=10 nb_inst_per_class=10']

In [74]:
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10',
 'all: n_random_points=10 nb_inst_per_class=10']

In [75]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_to_cd=df_perf[df_perf.classifier_name.isin(best_comb_by_method)]
df_to_cd.classifier_name.unique()

array(['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=10'], dtype=object)

In [76]:
draw_cd_diagram(df_to_cd, labels=True, title="Best combination comparison", fname='images_cd_diagram/cd-diagram_best_com.png')


the null hypothesis over the entire classifiers cannot be rejected


NameError: name 'exit' is not defined

In [ ]:
draw_cd_diagram(df_perf, labels=True, fname='images_cd_diagram/cd-diagram_all_methods.png')


#### Focus on most accurate lenght method

##### Overall Accuracy

In [ ]:
# highest accurate hyperparameters
best_comb_by_method

In [ ]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result
#df_result_acc=df_result_acc[df_result_acc["dataset_name"]=="Fungi"]

pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

In [ ]:
# create a pivot table with the variatioon of score by hyperparameter
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

##### Overall time complexity

In [ ]:
df_result_acc.time.sum()/(60*60*24)

In [ ]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='sum')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

In [ ]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

##### Calculate weights time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Finding subsequences time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Transform Dataset time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Classifier time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')